## NB-03 Information Retrieval

Now that I have selected my embeddings and added them to the postgreSQL database, we can now use PGVector to run our similarity searches to retrieve our desired chunks which will form the final stage of our RAG pipeline

What I want to do here is to generate the query prompt, and then feed it into the model - treat it as a chunk -> tokenise it -> generate the token-level embeddings vector -> alter the vector embeddings with context specific considerations -> amalgamate to produce a chunk level of embedding -> 

After the retrieval portion 
Reranking chunks before answering using a cross-encoder 


In [24]:
# Imports
import pandas as pd
from dotenv import load_dotenv
from sqlalchemy import text, create_engine
from sqlalchemy.orm import sessionmaker
import os
# Import the main components
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


# Imports from the 🤗 HuggingFace transformers library.
from transformers import AutoTokenizer, AutoModel

#loading the env file
load_dotenv()

# Create a connection to your database
engine = create_engine(os.getenv("DATABASE_URL"))

# Create a session
session = sessionmaker(engine)()

In [113]:
import sys
import os
import importlib

parent_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))
if parent_dir not in sys.path:
    sys.path.insert(0, parent_dir)

import w2v_pre_processing
importlib.reload(w2v_pre_processing)

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/bilalhashim/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/bilalhashim/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/bilalhashim/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


<module 'w2v_pre_processing' from '/Users/bilalhashim/Desktop/LSE/Year 4/DS205/summative_project_2/problem-set-2-BilalNHashim/w2v_pre_processing.py'>

In [3]:
with engine.connect() as conn:
    conn.execute(text("""
        CREATE INDEX IF NOT EXISTS doc_chunks_embedding_idx
        ON doc_chunks
        USING ivfflat (embedding vector_cosine_ops)
        WITH (lists = 100);
    """))
    conn.commit()

## Prompt Engineering

In this section I am trying to identify the most optimal set of prompts that I can use, we reintroduce the climate BERT model to embed the prompts

In [478]:
from w2v_pre_processing import get_pooled_embeddings
climate_model_name = 'climatebert/distilroberta-base-climate-f'
climate_tokeniser = AutoTokenizer.from_pretrained(climate_model_name)
climate_roberta_model = AutoModel.from_pretrained(climate_model_name)

def tokenize_for_transformer_climate_roberta(text):
    return climate_tokeniser(text, truncation=True, padding=True, return_tensors="pt")

/Users/bilalhashim/Desktop/LSE/Year 4/DS205/summative_project_2/problem-set-2-BilalNHashim/venv/lib/python3.12/site-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of RobertaModel were not initialized from the model checkpoint at climatebert/distilroberta-base-climate-f and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


This is our baseline query - I distilled down the most important parts of the project problem statement to construct a query which is relevant but not redundantly wordy

In [36]:
full_query_text = '“What emissions reduction target is each country in the NDC registry aiming for by 2030?”'
query_text = "What are the emissions reduction targets for 2030?"
tokenised_query = tokenize_for_transformer_climate_roberta(query_text)
query_embedding = get_pooled_embeddings(tokenised_query, climate_roberta_model)

embedding_str = f"[{','.join(str(x) for x in query_embedding)}]"

/Users/bilalhashim/Desktop/LSE/Year 4/DS205/summative_project_2/problem-set-2-BilalNHashim/venv/lib/python3.12/site-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of RobertaModel were not initialized from the model checkpoint at climatebert/distilroberta-base-climate-f and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [50]:
query = text("SELECT DISTINCT country FROM documents;")

with engine.connect() as conn:
    results = conn.execute(query).fetchall()

# Convert to list
country_list = [row[0] for row in results]
country_list

['Bangladesh',
 'Indonesia',
 'Brunei Darussalam',
 'Cameroon',
 'Kiribati',
 'Viet Nam',
 'Uganda',
 'Montenegro',
 'Jordan',
 'Syrian Arab Republic',
 'Dominican Republic',
 'Cambodia',
 'Papua New Guinea',
 'Singapore',
 'San Marino',
 'Sri Lanka',
 'Uzbekistan',
 'Finland',
 'Colombia',
 'United Kingdom of Great Britain and Northern Ireland',
 'Albania',
 'Grenada',
 'Ukraine',
 'Saudi Arabia',
 'Sao Tome and Principe',
 'Cuba',
 'Kyrgyzstan',
 'Algeria',
 'Maldives',
 'Djibouti',
 'Israel',
 'Nauru',
 'Kenya',
 'Senegal',
 'Malaysia',
 'Ghana',
 'Iceland',
 'Zambia',
 'Kuwait',
 'Madagascar',
 'Sierra Leone',
 'Bosnia and Herzegovina',
 'Liberia',
 'Philippines',
 'Benin',
 'Tuvalu',
 'State of Palestine',
 'Cabo Verde',
 'Guinea',
 'Nigeria',
 'Rwanda',
 'Tajikistan',
 'Zimbabwe',
 "Lao People's Democratic Republic",
 'China',
 'Comoros',
 'Saint Lucia',
 'Armenia',
 'Belarus',
 'Qatar',
 'Lesotho',
 'Paraguay',
 'Gabon',
 'Australia',
 'Dominica',
 'Mauritius',
 'Serbia',
 'Ango

Running our similarity search with PGVector based on the baseline query <BR> <BR>

This is the standard similarity search procedure I use for the remainder of the searches (explained in depth in the report.md file)

In [119]:
# Run similarity search
all_results = []
for country in country_list:
    query = text("""
        SELECT 
            doc_chunks.doc_id,
            doc_chunks.chunk_index,
            doc_chunks.content,
            documents.country,
            (doc_chunks.embedding <=> :embedding) AS similarity
        FROM doc_chunks
        LEFT JOIN documents ON doc_chunks.doc_id = documents.doc_id
        WHERE documents.country = :country
        ORDER BY similarity ASC
        LIMIT 10
    """)

    with engine.connect() as conn:
        results = conn.execute(query, {
            "embedding": embedding_str,
            "country": country
        }).fetchall()

# View results
        df_results = pd.DataFrame(results, columns=["doc_id", "chunk_index", "content", "country", "similarity"])
        df_results.rename(columns={"similarity": "cosine_distance"}, inplace=True)
        df_results["cosine_similarity"] = 1 - df_results["cosine_distance"]
        all_results.append(df_results)

final_df = pd.concat(all_results, ignore_index=True)

/var/folders/v6/xrbmfj054b7cb1_bvtvf02nc0000gn/T/ipykernel_65382/2490464773.py:30: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  final_df = pd.concat(all_results, ignore_index=True)


In [120]:
final_df

,doc_id,chunk_index,content,country,cosine_distance,cosine_similarity
0,bangladesh_english_20220601,450,Actions by 2030,Bangladesh,0.029974,0.970026
1,bangladesh_english_20220601,305,Actions by 2030,Bangladesh,0.029974,0.970026
2,bangladesh_english_20220601,366,Actions by 2030,Bangladesh,0.029974,0.970026
3,bangladesh_english_20220601,545,Actions by 2030,Bangladesh,0.029974,0.970026
4,bangladesh_english_20220601,502,Actions by 2030,Bangladesh,0.029974,0.970026
...,...,...,...,...,...,...
582,samoa_english_20220601,551,Selection of a base year,Samoa,0.048144,0.951856
583,antigua_and_barbuda_english_20220601,367,an indicative start year of 2025),Antigua and Barbuda,0.034179,0.965821
584,antigua_and_barbuda_english_20220601,226,for the period of 2020 - 2030,Antigua and Barbuda,0.047802,0.952198
585,somalia_english_20220601,198,"Estimated investment required (in USD, 2021-20...",Somalia,0.038558,0.961442


In [44]:
session.rollback()

Recall as well from the second notebook that we wanted to also consider words which flagged as being highly similar to the core-words from our query, as this might help to optimise the retrieval process

In [7]:
import pickle
with open("cumulative_tokens.pkl", "rb") as f:
    cumulative_similar_tokens_query = pickle.load(f)

cumulative_similar_tokens_query

{'emission': [('emissions', 0.8504191637039185),
  ('net', 0.6021032333374023),
  ('emisisons', 0.6007767915725708),
  ('reducfion', 0.6007478833198547),
  ('emitting', 0.5677352547645569),
  ('ghgs', 0.6484422087669373),
  ('concentration', 0.6424049139022827),
  ('possible', 0.6324024796485901),
  ('overall', 0.6215030550956726),
  ('double', 0.6183876395225525)],
 'reduction': [('reductions', 0.7689696550369263),
  ('reducing', 0.680912971496582),
  ('reduce', 0.57728511095047),
  ('figure', 0.5618992447853088),
  ('limitation', 0.5305630564689636),
  ('reducing', 0.7049707770347595),
  ('reduce', 0.6732713580131531),
  ('cut', 0.6551539897918701),
  ('reducfions', 0.6286653876304626),
  ('double', 0.6259312033653259)],
 'target': [('targets', 0.8495582342147827),
  ('updated', 0.7189120650291443),
  ('commitment', 0.7160426378250122),
  ('previous', 0.7054238319396973),
  ('revised', 0.6873401403427124),
  ('commitment', 0.7701578140258789),
  ('achieved', 0.7536922693252563),
  ('

From the above - I have qualitatively identified words that could be used to add context/content richness to our prompts - adding them to the below dict. To note, my embeddings will already be 'aware' of the similarity in these words, but including them can help to ensure a more in line response <BR> <BR>

Here I use a randomisation procedure to construct queries which should, meaningfully replicate the baseline query whilst testing for differences in inclusion of '2030' and '%' and 'percent'

In [159]:
import random 
similar_queries = []
content_additive_words = {
    'emission': ['net', 'ghgs', 'greenhouse'],
    'target': ['commitment', 'revised', 'goal', 'scope', 'objective'],
    'ndc': ['progress', 'step', 'submission'],
    'aiming': ['looking', 'seeking', 'aiming']
}
date_inclusion = [2030, '']
percent_inclusion = ['percent', '%', '']

for _ in range(10):
    emissions_replacement = random.choice(content_additive_words['emission'])
    target_replacement = random.choice(content_additive_words['target'])
    aiming_replacement = random.choice(content_additive_words['aiming'])
    date_replacement = random.choice(date_inclusion)
    percent_replacement = random.choice(percent_inclusion)

    similar_query = f'{emissions_replacement} {target_replacement} {aiming_replacement} {date_replacement} {percent_replacement}'.strip()
    if similar_query not in similar_queries:
        similar_queries.append(similar_query)
    
similar_queries


['ghgs commitment seeking 2030 %',
 'ghgs goal aiming  %',
 'greenhouse revised aiming  %',
 'greenhouse objective aiming',
 'net revised seeking',
 'greenhouse objective looking 2030 percent',
 'net goal aiming',
 'net revised seeking 2030 percent',
 'greenhouse scope looking  %']

The above looks to be sound in terms of words included and content. However, the logical coherence of these sentences is of course flawed because of our construction method <BR> <BR>

Below I generate embeddings for each of these queries for similarity search

In [160]:
similar_queries_embeddings = {}
for query in similar_queries:
    tokenised_query = tokenize_for_transformer_climate_roberta(query)
    query_embedding = get_pooled_embeddings(tokenised_query, climate_roberta_model)
    embedding_str = f"[{','.join(str(x) for x in query_embedding)}]"
    similar_queries_embeddings[query] = embedding_str

similar_queries_embeddings

{'ghgs commitment seeking 2030 %': '[-0.034325443,-0.08202738,-0.07436905,-0.0047721183,0.31819838,0.056827612,-0.04221918,0.08525371,0.034144875,-0.13183677,0.13390619,0.18829918,-0.028370287,-0.15103234,0.09272954,-0.018728342,-0.062804274,0.010321592,0.17176038,0.0138945775,-0.06454671,0.16930975,0.109424055,0.13421376,0.21931915,0.19386443,-0.08955947,-0.06738116,0.087019615,-0.25876868,0.05959017,-0.18063557,0.018344052,0.14599195,0.1666557,0.015410055,0.013845953,0.05855412,-0.14936659,0.11223957,0.112508446,-0.1328705,0.1245174,0.037790876,-0.08439667,0.123526305,0.17833059,0.19197145,0.048318014,-0.00763916,0.06860012,0.026593419,-0.054894883,0.043501243,0.023810243,-0.06658885,-0.049101118,0.1611738,-0.00654902,0.056822374,0.06325117,0.2079953,-0.11484671,0.2354936,-0.09605202,0.022060376,-0.041279398,0.23484312,0.021023504,0.06586919,0.11005714,-0.1359883,0.113078676,0.12169834,0.051708814,0.08147492,0.13351464,-3.3268423,-0.40531865,0.113179035,-0.022732303,0.007538042,0.492

Running Similarity Search for each of these queries for every country

In [161]:
all_results = []
for country in country_list:
    for sim_query, embedding_str in similar_queries_embeddings.items():
        query = text("""
        SELECT 
            doc_chunks.doc_id,
            doc_chunks.chunk_index,
            doc_chunks.content,
            documents.country,
            doc_chunks.climate_bert_cluster_labels,
            doc_chunks.w2v_cluster_labels,
            (doc_chunks.embedding <=> :embedding) AS cosine_distance
        FROM doc_chunks
        LEFT JOIN documents ON doc_chunks.doc_id = documents.doc_id
        WHERE 
            documents.country = :country
        ORDER BY cosine_distance ASC
        LIMIT 5
        """)
        params = {
        "embedding": embedding_str,
        }   

        with engine.connect() as conn:
            results = conn.execute(query, {
                "embedding": embedding_str,
                "country": country
            }).fetchall()

            # View results
            df_results = pd.DataFrame(results, columns=[
            "doc_id", "chunk_index", "content", "country", 
            "climate_bert_cluster", "w2v_cluster", "cosine_distance"
            ])
            df_results["sim_query"] = sim_query
            all_results.append(df_results)

similar_queries_df = pd.concat(all_results, ignore_index=True)

/var/folders/v6/xrbmfj054b7cb1_bvtvf02nc0000gn/T/ipykernel_65382/890052353.py:38: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  similar_queries_df = pd.concat(all_results, ignore_index=True)


In [162]:
similar_queries_df

,doc_id,chunk_index,content,country,climate_bert_cluster,w2v_cluster,cosine_distance,sim_query
0,bangladesh_english_20220601,541,Ensure 3R principle for waste,Bangladesh,"Headers, Keywords, Multilingual",Mitigation & Adaptation Strategies – English,0.031867,ghgs commitment seeking 2030 %
1,bangladesh_english_20220601,421,Ensure 3R principle for waste,Bangladesh,"Headers, Keywords, Multilingual",Mitigation & Adaptation Strategies – English,0.031867,ghgs commitment seeking 2030 %
2,bangladesh_english_20220601,512,(generating 164 MW) for agriculture,Bangladesh,"Headers, Keywords, Multilingual",Project Sectors & Technologies,0.039676,ghgs commitment seeking 2030 %
3,bangladesh_english_20220601,473,(ERP) or congestion,Bangladesh,"Headers, Keywords, Multilingual",Fragmented Phrases & Minor Details,0.041617,ghgs commitment seeking 2030 %
4,bangladesh_english_20220601,341,(ERP) or congestion,Bangladesh,"Headers, Keywords, Multilingual",Fragmented Phrases & Minor Details,0.041617,ghgs commitment seeking 2030 %
...,...,...,...,...,...,...,...,...
4837,andorra_spanish_20250201,888,Impacto,Andorra,"Headers, Keywords, Multilingual",Formal Climate Pledges – Spanish,0.046013,net goal aiming
4838,andorra_spanish_20250201,38,"Business as usual, escenario inmovilista",Andorra,"Headers, Keywords, Multilingual",Fragmented Phrases & Minor Details,0.048813,net revised seeking 2030 percent
4839,andorra_spanish_20230101,50,Business as usual,Andorra,"Headers, Keywords, Multilingual",Emission Reduction Targets & Commitments,0.076005,net revised seeking 2030 percent
4840,andorra_spanish_20250201,38,"Business as usual, escenario inmovilista",Andorra,"Headers, Keywords, Multilingual",Fragmented Phrases & Minor Details,0.063279,greenhouse scope looking %


In [163]:
similar_queries_embeddings

{'ghgs commitment seeking 2030 %': '[-0.034325443,-0.08202738,-0.07436905,-0.0047721183,0.31819838,0.056827612,-0.04221918,0.08525371,0.034144875,-0.13183677,0.13390619,0.18829918,-0.028370287,-0.15103234,0.09272954,-0.018728342,-0.062804274,0.010321592,0.17176038,0.0138945775,-0.06454671,0.16930975,0.109424055,0.13421376,0.21931915,0.19386443,-0.08955947,-0.06738116,0.087019615,-0.25876868,0.05959017,-0.18063557,0.018344052,0.14599195,0.1666557,0.015410055,0.013845953,0.05855412,-0.14936659,0.11223957,0.112508446,-0.1328705,0.1245174,0.037790876,-0.08439667,0.123526305,0.17833059,0.19197145,0.048318014,-0.00763916,0.06860012,0.026593419,-0.054894883,0.043501243,0.023810243,-0.06658885,-0.049101118,0.1611738,-0.00654902,0.056822374,0.06325117,0.2079953,-0.11484671,0.2354936,-0.09605202,0.022060376,-0.041279398,0.23484312,0.021023504,0.06586919,0.11005714,-0.1359883,0.113078676,0.12169834,0.051708814,0.08147492,0.13351464,-3.3268423,-0.40531865,0.113179035,-0.022732303,0.007538042,0.492

In [164]:
bool_mask = similar_queries_df['sim_query'] == "ghgs commitment seeking 2030 %"
for row in similar_queries_df[bool_mask]['content']:
    print(row)

Ensure 3R principle for waste
Ensure 3R principle for waste
(generating 164 MW) for agriculture
(ERP) or congestion
(ERP) or congestion
iii) For climate forcers
ii) Specific projects, measures
ii) Contextual matters,
c) Other contextual
Tools used:
 12% inconditionnel
a) Mesures inconditionnelles ................................................................... 19
Type de contribution
(23%) à
CDN CH4 CMA
Note (*): increasing GHG removal
43.5 Note (*): increasing GHG removal
to reducing
but without many specific results.
Metric applied
Outcome: Increased- resilient Fisheries sector
Area under agroforestr y landscape (ha)
Forest landscape restored (ha)
change in Uganda. Final Report.
this waste needs
GDP growth in %
E. Prescribed burning of savannahs
E. Prescribed burning of savannahs
E. Prescribed burning of savannahs
E. Prescribed burning of savannahs
o Energy Efficiency Projects in industry:
Measures in Residential sector:
o o o Energy measures in Residential sector
augmentation to


The above shows that when we omit semantic sentence structuring we get poorly returned content chunks. Likely because the embeddings match those headings or table entries which have a somewhat unnatural phrasing structure. <BR>

Whilst interesting as an academic excercise, constructing queries based PURELY on Word2Vec word similarity does not seem to be, because it ignores transformer models ability to pick up on semantic context <BR>

__NOTE:__ I manually looked through the rest of the retrieved content blocks from the other 'W2V randomly constructed' queries -> They yielded similar poor results

After exploration below, I build upon this approach by using my retrieved 'gold-standard' ground truth retrieved content blocks to generate a vector of queries which: (A) Allign with the W2V Keywords, (B) Follow natural sentence structure, and (B) Stick closely to those ground truth returned answers

### Search Metric Comparison

Here I decide to compare search metrics, using the same procedure to qualitatively analyse chunks returned by top-K approach for: cosine similarity, l2 distance and inner product <BR> <BR>

__cosine similarity__

In [92]:
all_results = []
for country in country_list:
    query = text("""
    SELECT 
        doc_chunks.doc_id,
        doc_chunks.chunk_index,
        doc_chunks.content,
        documents.country,
        doc_chunks.climate_bert_cluster_labels,
        doc_chunks.w2v_cluster_labels,
        (doc_chunks.embedding <-> :embedding) AS cosine_distance
    FROM doc_chunks
    LEFT JOIN documents ON doc_chunks.doc_id = documents.doc_id
    WHERE 
        documents.country = :country
    ORDER BY cosine_distance ASC
    LIMIT 5
    """)
    params = {
    "embedding": embedding_str
    }   

    with engine.connect() as conn:
        results = conn.execute(query, {
            "embedding": embedding_str,
            "country": country
        }).fetchall()

        # View results
        df_results = pd.DataFrame(results, columns=[
        "doc_id", "chunk_index", "content", "country", 
        "climate_bert_cluster", "w2v_cluster", "cosine_distance"
        ])
        all_results.append(df_results)

cosine_comparative_df = pd.concat(all_results, ignore_index=True)

/var/folders/v6/xrbmfj054b7cb1_bvtvf02nc0000gn/T/ipykernel_65382/2563596234.py:36: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  cosine_comparative_df = pd.concat(all_results, ignore_index=True)


__L2 Distance__

In [93]:
all_results = []
for country in country_list:
    query = text("""
    SELECT 
        doc_chunks.doc_id,
        doc_chunks.chunk_index,
        doc_chunks.content,
        documents.country,
        doc_chunks.climate_bert_cluster_labels,
        doc_chunks.w2v_cluster_labels,
        (doc_chunks.embedding <-> :embedding) AS l2_distance
    FROM doc_chunks
    LEFT JOIN documents ON doc_chunks.doc_id = documents.doc_id
    WHERE 
        documents.country = :country
    ORDER BY l2_distance ASC
    LIMIT 5
    """)
    params = {
    "embedding": embedding_str
    }   

    with engine.connect() as conn:
        results = conn.execute(query, {
            "embedding": embedding_str,
            "country": country
        }).fetchall()

        # View results
        df_results = pd.DataFrame(results, columns=[
        "doc_id", "chunk_index", "content", "country", 
        "climate_bert_cluster", "w2v_cluster", "l2_distance"
        ])
        all_results.append(df_results)

l2_comparative_df = pd.concat(all_results, ignore_index=True)

/var/folders/v6/xrbmfj054b7cb1_bvtvf02nc0000gn/T/ipykernel_65382/1327533772.py:36: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  l2_comparative_df = pd.concat(all_results, ignore_index=True)


__Inner Product__

In [94]:
all_results = []
for country in country_list:
    query = text("""
    SELECT 
        doc_chunks.doc_id,
        doc_chunks.chunk_index,
        doc_chunks.content,
        documents.country,
        doc_chunks.climate_bert_cluster_labels,
        doc_chunks.w2v_cluster_labels,
        (doc_chunks.embedding <#> :embedding) AS inner_product
    FROM doc_chunks
    LEFT JOIN documents ON doc_chunks.doc_id = documents.doc_id
    WHERE 
        documents.country = :country
    ORDER BY inner_Product ASC
    LIMIT 5
    """)
    params = {
    "embedding": embedding_str,
    }   

    with engine.connect() as conn:
        results = conn.execute(query, {
            "embedding": embedding_str,
            "country": country
        }).fetchall()

        # View results
        df_results = pd.DataFrame(results, columns=[
        "doc_id", "chunk_index", "content", "country", 
        "climate_bert_cluster", "w2v_cluster", "inner_product"
        ])
        all_results.append(df_results)

inner_product_comparative_df = pd.concat(all_results, ignore_index=True)

/var/folders/v6/xrbmfj054b7cb1_bvtvf02nc0000gn/T/ipykernel_65382/3336980235.py:36: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  inner_product_comparative_df = pd.concat(all_results, ignore_index=True)


In [89]:
def prepare_distance_df(df, distance_column, method_name):
    df = df.copy()
    df["distance_metric"] = method_name
    df = df.rename(columns={distance_column: "distance_value"})
    df["rank"] = df.groupby("country")["distance_value"].rank(method="first")
    return df[[
        "country", "rank", "distance_metric", "doc_id", "chunk_index", "content",
        "climate_bert_cluster", "w2v_cluster", "distance_value"
    ]]

In [95]:
df_cosine_prepped = prepare_distance_df(cosine_comparative_df, "cosine_distance", "cosine")
df_l2_prepped = prepare_distance_df(l2_comparative_df, "l2_distance", "l2")
df_inner_prepped = prepare_distance_df(inner_product_comparative_df, "inner_product", "inner_product")

In [96]:
final_comparison_df = pd.concat(
    [df_cosine_prepped, df_l2_prepped, df_inner_prepped],
    ignore_index=True
)

##### Now we can make comaprisons for different countries to see which returns the most intuitively correct response for our query

In [103]:
boolean_mask = final_comparison_df['country'] == 'Bangladesh'
bangladesh_comparative_df = final_comparison_df[boolean_mask]

bangladesh_comparative_df

,country,rank,distance_metric,doc_id,chunk_index,content,climate_bert_cluster,w2v_cluster,distance_value
0,Bangladesh,1.0,cosine,bangladesh_english_20220601,770,Single-year targets for 2030. The target might...,Narrative - English,Emission Reduction Targets & Commitments,3.101959
1,Bangladesh,2.0,cosine,bangladesh_english_20220601,745,Total GHG emission will be 409.4 MtCO2e in 203...,Narrative - English,Emission Reduction Targets & Commitments,3.205943
2,Bangladesh,3.0,cosine,bangladesh_english_20220601,87,Critical assumptions for modelling GHG emissio...,Narrative - English,Emission Reduction Targets & Commitments,3.355135
3,Bangladesh,4.0,cosine,bangladesh_english_20220601,300,The targeted GHG emission reduction for uncond...,Narrative - English,Emission Reduction Targets & Commitments,3.409312
4,Bangladesh,5.0,cosine,bangladesh_english_20220601,26,The following table 1 presents the sector-wise...,Narrative - English,Emission Reduction Targets & Commitments,3.540240
785,Bangladesh,1.0,l2,bangladesh_english_20220601,770,Single-year targets for 2030. The target might...,Narrative - English,Emission Reduction Targets & Commitments,3.101959
786,Bangladesh,2.0,l2,bangladesh_english_20220601,745,Total GHG emission will be 409.4 MtCO2e in 203...,Narrative - English,Emission Reduction Targets & Commitments,3.205943
787,Bangladesh,3.0,l2,bangladesh_english_20220601,87,Critical assumptions for modelling GHG emissio...,Narrative - English,Emission Reduction Targets & Commitments,3.355135
788,Bangladesh,4.0,l2,bangladesh_english_20220601,300,The targeted GHG emission reduction for uncond...,Narrative - English,Emission Reduction Targets & Commitments,3.409312
789,Bangladesh,5.0,l2,bangladesh_english_20220601,26,The following table 1 presents the sector-wise...,Narrative - English,Emission Reduction Targets & Commitments,3.540240


At face value, referring to our clustering strategy all seme to be pulling from the most relevant clusters that we had identified. However, inner product seems to be pulling chunks from different clusters. Cosine similarity and l2 distance seem to reach consensus that the most relevant chunks were those previously clustered under 'Emission Reduction Targets & Commitments'. This also is the most likely relevant cluster which suggests positively in favour of these two distance measures 

#### Multi-Country Analysis for each method 

__Bangladesh__

In [106]:
bangladesh_comparative_df = bangladesh_comparative_df.reset_index(drop=True)

for i in range(len(bangladesh_comparative_df['content'])):
    if i == 0:
        print('=' * 50)
        print('COSINE SIMILARITY RESULTS')
        print('=' * 50)
    elif i == 5:
        print('=' * 50)
        print('L2 DISTANCE RESULTS')
        print('=' * 50)
    elif i == 10:
        print('=' * 50)
        print('INNER PRODUCT RESULTS')
        print('=' * 50)
    print(bangladesh_comparative_df['content'][i])
    

COSINE SIMILARITY RESULTS
Single-year targets for 2030. The target might be updated in 2025.
Total GHG emission will be 409.4 MtCO2e in 2030 under BAU scenario. The contributions of the sectors are;
Critical assumptions for modelling GHG emission for BAU scenarios are:
The targeted GHG emission reduction for unconditional contributions will be implemented through a set of mitigation actions. The potential mitigations actions are elaborated in Table 4.
The following table 1 presents the sector-wise GHG emission for the Base year scenario.
L2 DISTANCE RESULTS
Single-year targets for 2030. The target might be updated in 2025.
Total GHG emission will be 409.4 MtCO2e in 2030 under BAU scenario. The contributions of the sectors are;
Critical assumptions for modelling GHG emission for BAU scenarios are:
The targeted GHG emission reduction for unconditional contributions will be implemented through a set of mitigation actions. The potential mitigations actions are elaborated in Table 4.
The fo

__Angola__

In [109]:
boolean_mask = final_comparison_df['country'] == 'Angola'
angola_comparative_df = final_comparison_df[boolean_mask]

angola_comparative_df

angola_comparative_df = angola_comparative_df.reset_index(drop=True)

for i in range(len(angola_comparative_df['content'])):
    if i == 0:
        print('=' * 50)
        print('COSINE SIMILARITY RESULTS')
        print('=' * 50)
    elif i == 5:
        print('=' * 50)
        print('L2 DISTANCE RESULTS')
        print('=' * 50)
    elif i == 10:
        print('=' * 50)
        print('INNER PRODUCT RESULTS')
        print('=' * 50)
    print(angola_comparative_df['content'][i])

COSINE SIMILARITY RESULTS
At least 14% domestic reduction in greenhouse gas emissions by 2025.
Angola is setting the target to achieve a 14% reduction (unconditionally) by 2025.
Additional 10% of GHG reduction by 2025
In terms of flaring, we have increased the ambition to reduce flaring. Instead of 75% reduction, as planned and presented in the unconditional 2030 scenario, it is now considered a 91% emission reduction compared to 2015 emissions.
implications for emissions in the transport sector, resulting in a rise in global temperature and climate change.
L2 DISTANCE RESULTS
At least 14% domestic reduction in greenhouse gas emissions by 2025.
Angola is setting the target to achieve a 14% reduction (unconditionally) by 2025.
Additional 10% of GHG reduction by 2025
In terms of flaring, we have increased the ambition to reduce flaring. Instead of 75% reduction, as planned and presented in the unconditional 2030 scenario, it is now considered a 91% emission reduction compared to 2015 emi

__Jordan__

In [110]:
boolean_mask = final_comparison_df['country'] == 'Jordan'
jordan_comparative_df = final_comparison_df[boolean_mask]

jordan_comparative_df

jordan_comparative_df = jordan_comparative_df.reset_index(drop=True)

for i in range(len(jordan_comparative_df['content'])):
    if i == 0:
        print('=' * 50)
        print('COSINE SIMILARITY RESULTS')
        print('=' * 50)
    elif i == 5:
        print('=' * 50)
        print('L2 DISTANCE RESULTS')
        print('=' * 50)
    elif i == 10:
        print('=' * 50)
        print('INNER PRODUCT RESULTS')
        print('=' * 50)
    print(jordan_comparative_df['content'][i])

COSINE SIMILARITY RESULTS
The target is a single year for 2030
8. Reducing emissions that are under the direct control of the health sector to reach net zero health sector emissions by 2050.
The energy measures that are considered to achieve the NDC target are;
Jordan sets the target of achieving 31% reduction of emissions by 2030. Which is quantified to be 11269 Gg CO2 eq.
Energy measures including renewable energy and energy efficiency targets up to 2030 were considered and estimated, namely;
L2 DISTANCE RESULTS
The target is a single year for 2030
8. Reducing emissions that are under the direct control of the health sector to reach net zero health sector emissions by 2050.
The energy measures that are considered to achieve the NDC target are;
Jordan sets the target of achieving 31% reduction of emissions by 2030. Which is quantified to be 11269 Gg CO2 eq.
Energy measures including renewable energy and energy efficiency targets up to 2030 were considered and estimated, namely;
INNER 

The above 3 country tests shows great similarity between the L2 Distance results and the cosine similarity results. This is to be expected given that the calculus for each is done in similar ways. For the Inner Product results, the calculation method here seems to favour more explantory context giving statements, which is very interesting. For the purpose of our task. <BR>

__However__, I recognise if our task was to do something like: 'find the emissions target for xyz country and give context for why they have set thsis target and how they will achieve, their methodology etc.' then I would most probably construct my prompt to the LLM based on top k-results for BOTH Cosine similarity distances and INNER PRODUCT, as this would allow the LLM to generate a response both empirically accurate and qualitatively grounded

As it stands though, cosine similarity fits the brief of this task the most accurately and so I use it going forward

In [172]:
del final_df

#### Re-running our query procedure to update the dataframe - this time Selecting cluster labels to add to our dataframe

In [175]:
query_text = "What are the emissions reduction targets for 2030?"
tokenised_query = tokenize_for_transformer_climate_roberta(query_text)
query_embedding = get_pooled_embeddings(tokenised_query, climate_roberta_model)

embedding_str = f"[{','.join(str(x) for x in query_embedding)}]"

In [289]:
all_results = []
for country in country_list:
    query = text("""
    SELECT 
        doc_chunks.doc_id,
        doc_chunks.chunk_index,
        doc_chunks.content,
        documents.country,
        doc_chunks.climate_bert_cluster_labels,
        doc_chunks.w2v_cluster_labels,
        doc_chunks.chunk_metadata,
        (doc_chunks.embedding <=> :embedding) AS cosine_distance
    FROM doc_chunks
    LEFT JOIN documents ON doc_chunks.doc_id = documents.doc_id
    WHERE 
        documents.country = :country
        AND doc_chunks.content ILIKE '%2030%'
        AND (doc_chunks.embedding <=> :embedding) <= :distance_threshold
    ORDER BY cosine_distance ASC
    LIMIT 10
    """)
    params = {
    "embedding": embedding_str,
    "distance_threshold": 0.05
    }   

    with engine.connect() as conn:
        results = conn.execute(query, {
            "embedding": embedding_str,
            "country": country,
            "distance_threshold": 0.05
        }).fetchall()

        # View results
        df_results = pd.DataFrame(results, columns=[
        "doc_id", "chunk_index", "content", "country", 
        "climate_bert_cluster", "w2v_cluster", "chunk_metadata", "cosine_distance"
        ])
        df_results["cosine_similarity"] = 1 - df_results["cosine_distance"]
        all_results.append(df_results)

final_df = pd.concat(all_results, ignore_index=True)

/var/folders/v6/xrbmfj054b7cb1_bvtvf02nc0000gn/T/ipykernel_65382/2563391061.py:42: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  final_df = pd.concat(all_results, ignore_index=True)


In [290]:
final_df[10:20]

for row in final_df[10:20]['content']:
    print(row)

sectoral development plans for the period 2021-2030 towards the goals of the National Climate Change Strategy to 2050. Specific measures to reduce GHG emissions include:
Emission reduction targets in the energy, agriculture, LULUCF, waste and industrial processes by 2030 compared to BAU in NDC 2022 are higher than NDC 2020, Unconditional contribution increased from 9% to 15.8% and Conditional contribution increased from 27% to 43.5%.
The updated NDC highlights a huge progression from 22% to 24.7% reduction in 2030.
Full implementation of this measure will to emissions reduce potentially approximately 2.9 MtCO2e by 2030.
The NAMA seeks to contribute to GHG mitigation of 0.5 MtCO2e by 2030.
Economy wide emissions reduction of 24.7% in 2030 below the BAU conditions. Of which, Uganda’s unconditional efforts will result into reduction of 5.9% in 2030 below the BAU conditions.
Full implementation of the measure will contribute to a GHG emission reduction of approximately 3.37 MtCO2e by 2030.

In [291]:
final_df

,doc_id,chunk_index,content,country,climate_bert_cluster,w2v_cluster,chunk_metadata,cosine_distance,cosine_similarity
0,bangladesh_english_20220601,745,Total GHG emission will be 409.4 MtCO2e in 203...,Bangladesh,Narrative - English,Emission Reduction Targets & Commitments,"{'links': [], 'filename': 'bangladesh_english_...",0.024598,0.975402
1,bangladesh_english_20220601,10,Bangladesh’s INDC proposed for 12 million tons...,Bangladesh,Narrative - English,Emission Reduction Targets & Commitments,"{'links': [], 'filename': 'bangladesh_english_...",0.036378,0.963622
2,bangladesh_english_20220601,92,Total GHG emission is found to be increased fr...,Bangladesh,Narrative - English,Emission Reduction Targets & Commitments,"{'links': [], 'filename': 'bangladesh_english_...",0.040419,0.959581
3,bangladesh_english_20220601,588,"Under this comprehensive plan, the government ...",Bangladesh,Narrative - English,Emission Reduction Targets & Commitments,"{'links': [], 'filename': 'bangladesh_english_...",0.042267,0.957733
4,bangladesh_english_20220601,886,The updated unconditional and conditional cont...,Bangladesh,Narrative - English,Emission Reduction Targets & Commitments,"{'links': [], 'filename': 'bangladesh_english_...",0.044091,0.955909
...,...,...,...,...,...,...,...,...,...
559,antigua_and_barbuda_english_20220601,673,86% renewable energy generation in the electri...,Antigua and Barbuda,Narrative - English,Project Sectors & Technologies,"{'links': [], 'filename': 'antigua_and_barbuda...",0.032328,0.967672
560,antigua_and_barbuda_english_20220601,293,1. 86% renewable energy generation from local ...,Antigua and Barbuda,Narrative - English,Project Sectors & Technologies,"{'links': [], 'filename': 'antigua_and_barbuda...",0.035110,0.964890
561,antigua_and_barbuda_english_20220601,674,100% all new vehicle sales to be electric vehi...,Antigua and Barbuda,Narrative - English,Project Sectors & Technologies,"{'links': [], 'filename': 'antigua_and_barbuda...",0.036758,0.963242
562,antigua_and_barbuda_english_20220601,409,"10. 30,000 homes or 50% of pre-2020 homes to h...",Antigua and Barbuda,Narrative - English,Project Sectors & Technologies,"{'links': [], 'filename': 'antigua_and_barbuda...",0.047449,0.952551


# Establishing Ground-Truths/Gold-standard queries
The above querying system seems to work well, however to improve I wanted to 'train' my query on several return statements that have been __verified__ and that we know to contain the truths we want to obtain

In [197]:
aus_bool = final_df['country'] == 'Australia'

for i in range(len(final_df[aus_bool]['content'])):
    if i in [3,4,7, 8]:
        print('=' * 40 + '  GROUND TRUTH  ' + '=' * 80)
        print(final_df[aus_bool]['content'].iloc[i])
        print('=' * 140 + '\n')
    else:
        print(final_df[aus_bool]['content'].iloc[i] + '\n')

Australia’s 2030 and 2050 targets are economy-wide absolute emissions reduction targets.

Absolute economy-wide emissions reduction, as an emissions budget covering 2021-2030.

Australia assesses progress towards its 2030 target by comparing cumulative net emissions over the period 2021–2030 with the emissions budget for the period.

========================================  GROUND TRUTH  ================================================================================
Australia’s 2030 target is a 43% reduction below 2005 levels (reference indicator) by 2030, implemented as a single-year point target.

========================================  GROUND TRUTH  ================================================================================
Australia’s 2030 target is a 43% reduction below 2005 levels by 2030, implemented as an emissions budget (reference indicator) covering the period 2021-2030.

Implemented as an emissions budget covering the period 2021-2030

A commitment to reduce the em

__Here I manually went into the docs to verify that the returned chunk was indeed correct for identifying the target__ See SS's below

![Chart](ground_truth_images/Screenshot%202025-04-10%20at%2017.22.44.png)


![Chart](ground_truth_images/Screenshot%202025-04-10%20at%2017.24.31.png)


We see in Australia's case for example, that our model has picked up on 4 'target-relevant chunks' out of 10 of the returned content chunks, already this is very promising, but I want to see if we can improve our selection (e.g. Better ranking of the true emissions targets, increased percentage of top 10 chunks containing target if possible). <BR> 

I want to develop a general array which captures the various forms that the emissions target are presented in - we see just in Australia's case there are two or 3 distinct ways that the targets are presented

In [205]:
# I'm building up a list of the ground truth sentences that I can use to strengthen the final queries I use to retrieve the chunks that will go into the LLM
gold_standard = ['43% below 2005 levels by 2030', 
                 'Australia’s 2030 target is a 43% reduction below 2005 levels (reference indicator) by 2030, implemented as a single-year point target.',
                 'In this updated NDC, Australia is increasing the ambition of its 2030 target, committing to reduce greenhouse gas emissions 43% below 2005 levels by 2030.'
                 ]

##### Jordan 

In [201]:
jord_bool = final_df['country'] == 'Jordan'

for i in range(len(final_df[jord_bool]['content'])):
    if i in [0, 1, 4]:
        print('=' * 40 + '  GROUND TRUTH  ' + '=' * 80)
        print(final_df[jord_bool]['content'].iloc[i])
        print('=' * 140 + '\n')
    else:
        print(final_df[jord_bool]['content'].iloc[i] + '\n')

========================================  GROUND TRUTH  ================================================================================
Jordan sets the target of achieving 31% reduction of emissions by 2030. Which is quantified to be 11269 Gg CO2 eq.

========================================  GROUND TRUTH  ================================================================================
Outcomes of the analysis of the actions resulted in the target of total GHG emission reduction of 31% in 2030 compared to BAU in 2012 (description of actions is in Annex 1)

 The projected emissions under BAU scenario is estimated to 43,989.0 Gg CO2 eq in year 2030.

Increased percentage of electricity generated from renewables1 to have a share of more 35% by the year 2030

========================================  GROUND TRUTH  ================================================================================
2012 2015 2020 2025 2030 *The Estimated Mitigation Target is 31% ** Resulting target was calcul

<img src="ground_truth_images/Screenshot%202025-04-10%20at%2017.42.56.png" width="500"/>


__Other screenshots for Jordan and following countries omitted for brevity of the notebook__

For Jordan we see that we have 3 retrieved truths - all in different forms - again building up our 'gold-standard' templates

In [206]:
gold_standard.extend(['Jordan sets the target of achieving 31% reduction of emissions by 2030. Which is quantified to be 11269 Gg CO2 eq.', 'The Estimated Mitigation Target is 31%'])

In [212]:
final_df['country'].unique()

array(['Bangladesh', 'Brunei Darussalam', 'Cameroon', 'Viet Nam',
       'Uganda', 'Montenegro', 'Jordan', 'Cambodia', 'Papua New Guinea',
       'Singapore', 'Sri Lanka', 'Uzbekistan', 'Finland',
       'United Kingdom of Great Britain and Northern Ireland', 'Albania',
       'Grenada', 'Ukraine', 'Saudi Arabia', 'Sao Tome and Principe',
       'Cuba', 'Kyrgyzstan', 'Algeria', 'Maldives', 'Djibouti',
       'Malaysia', 'Ghana', 'Iceland', 'Zambia', 'Bosnia and Herzegovina',
       'Liberia', 'Philippines', 'Tuvalu', 'State of Palestine',
       'Cabo Verde', 'Rwanda', 'Tajikistan', 'Zimbabwe',
       "Lao People's Democratic Republic", 'China', 'Saint Lucia',
       'Armenia', 'Belarus', 'Australia', 'Dominica', 'Serbia', 'Angola',
       'Vanuatu', 'United Arab Emirates', 'Georgia', 'Bhutan',
       'Republic of Korea', 'Solomon Islands', 'Thailand', 'Togo',
       'Burkina Faso', 'Oman', 'Micronesia (Federated States of)', 'Fiji',
       'Holy See', 'Eritrea', 'Canada', 'Barbados',


In [217]:
swiss_bool = final_df['country'] == 'Switzerland'

for i in range(len(final_df[swiss_bool]['content'])):
    if i in [5]:
        print('=' * 40 + '  GROUND TRUTH  ' + '=' * 80)
        print(final_df[swiss_bool]['content'].iloc[i])
        print('=' * 140 + '\n')
    else:
        print(final_df[swiss_bool]['content'].iloc[i] + '\n')

globally, in particular methane emissions by 2030

Switzerland’s emission reductions by 2030 will mainly be achieved domestically, thereby further strengthening Switzerland’s transition to a low carbon economy. Given the low greenhouse gas intensity of Switzerland today, its first NDC represents a high level of ambition.

Switzerland expresses its first NDC both as single-year (2030) and multi-year target (2021–2030). The single-year target is implemented using an emission budget over the period 2021–2030.

about 45 per cent compared with 2010 levels by 2030, given Switzerland’s per capita emissions are below global average. On the other hand, and in line with the IPCC report on 1.5 degrees Celsius, the indicative target for 2050 communicated in 2015 has been revised and raised to net zero greenhouse gas emissions.

Emission reduction of at least minus 50 per cent by 2030 compared with 1990 levels, implemented as an emission budget covering 2021–2030, which is equivalent to an average 

In [218]:
gold_standard.extend(['Switzerland’s NDC target is to reduce greenhouse gas emissions by 50% by 2030 compared to 1990 levels.'])

In [221]:
belarus_bool = final_df['country'] == 'Belarus'

for i in range(len(final_df[belarus_bool]['content'])):
    if i in [4,5]:
        print('=' * 40 + '  GROUND TRUTH  ' + '=' * 80)
        print(final_df[belarus_bool]['content'].iloc[i])
        print('=' * 140 + '\n')
    else:
        print(final_df[belarus_bool]['content'].iloc[i] + '\n')

The type of commitment is the absolute reduction of greenhouse gas emissions until 2030 as compared to the base year emissions, inclusive of the LULUCF sector.

The year 2018 was chosen as the base year for generating emission projections and accounting for greenhouse gas reduction policy and measures until 2030.

Figure 1. Greenhouse gas emissions in 1990-2018 and commitment until 2030

The dynamics of greenhouse gas emissions exclusive of the LULUCF sector during 1990-2018 and the commitment until 2030 are shown in Figure 1.

========================================  GROUND TRUTH  ================================================================================
Taking into account the economic situation and the country's capabilities, the new unconditional economy-wide target is to reduce greenhouse gas emissions by at least 35 per cent from the 1990 level by 2030, inclusive of the LULUCF sector.

========================================  GROUND TRUTH  ================================

In [223]:
gold_standard.extend(['Belarus aims to reduce its greenhouse gas emissions by 2030 by 35% compared to 1990 levels.', 'The new conditional economy-wide target is to reduce greenhouse gas emissions by at least 40 per cent from the 1990 level by 2030'])

In [224]:
gold_standard

['43% below 2005 levels by 2030',
 'Australia’s 2030 target is a 43% reduction below 2005 levels (reference indicator) by 2030, implemented as a single-year point target.',
 'In this updated NDC, Australia is increasing the ambition of its 2030 target, committing to reduce greenhouse gas emissions 43% below 2005 levels by 2030.',
 'Jordan sets the target of achieving 31% reduction of emissions by 2030. Which is quantified to be 11269 Gg CO2 eq.',
 'The Estimated Mitigation Target is 31%',
 'Switzerland’s NDC target is to reduce greenhouse gas emissions by 50% by 2030 compared to 1990 levels.',
 'Belarus aims to reduce its greenhouse gas emissions by 2030 by 35% compared to 1990 levels.',
 'The new conditional economy-wide target is to reduce greenhouse gas emissions by at least 40 per cent from the 1990 level by 2030']

__We have now collected a nice list of 'gold-standard' phrases which capture exactly what we want to extract from the text - my idea is to curate a list of queries based on these and then generate embeddings for each of these, using them to fortify the chance that correct target-specific chunks are amongst the those that I feed into the LLM__

To do so, I need to generalise the statements so that they are country-neutral, remove the numbers, and overall clean up so that they are more simplified which will help make them queryable

In [250]:
gold_standard_queries = [
 '2030 target is a % reduction in ghgs below levels by 2030',
 'reduce greenhouse gas emissions % below levels by 2030.',
 '% reduction of GHG emissions by 2030. Which is quantified to be Gg CO2 eq.',
 'The Estimated Mitigation Target is X% by 2030',
 'NDC target is to reduce greenhouse gas emissions by % by 2030',
 'aims to reduce its greenhouse gas emissions by 2030 by per cent compared to levels.']

gold_standard_queries

['2030 target is a % reduction in ghgs below levels by 2030',
 'reduce greenhouse gas emissions % below levels by 2030.',
 '% reduction of GHG emissions by 2030. Which is quantified to be Gg CO2 eq.',
 'The Estimated Mitigation Target is X% by 2030',
 'NDC target is to reduce greenhouse gas emissions by % by 2030',
 'aims to reduce its greenhouse gas emissions by 2030 by per cent compared to levels.']

In [255]:
gold_standard_query_embeddings = {}
for query in gold_standard_queries:
    tokenised_gs_query = tokenize_for_transformer_climate_roberta(query)
    gs_query_embedding = get_pooled_embeddings(tokenised_gs_query, climate_roberta_model)
    embedding_string = f"[{','.join(str(x) for x in gs_query_embedding)}]"
    assert gs_query_embedding.shape == (768,)
    gold_standard_query_embeddings[query] = embedding_string

gold_standard_query_embeddings


{'2030 target is a % reduction in ghgs below levels by 2030': '[-0.083088815,-0.07100105,0.050188564,0.05355295,-0.08342649,-0.08223077,-0.008635866,-0.0027382877,-0.010806341,-0.099825874,0.07770924,0.06584646,-0.06461507,-0.23641361,-0.026772272,-0.13401426,-0.14741735,-0.041601434,0.03565541,0.14728871,-0.14270419,0.1776749,-0.010235806,0.11377233,0.29756594,0.08799892,-0.15508524,0.002043062,-0.078664325,-0.17776322,-0.004701345,-0.27038106,-0.027854282,0.20357004,0.009344762,0.2085027,0.21190171,0.017625496,0.075399205,0.18257661,0.19056778,-0.07406568,0.08521091,0.03487818,-0.11718399,0.14397494,0.20026255,0.22715232,0.014779244,0.03216515,0.13800967,0.032215506,-0.05575748,-0.005894262,0.0011886349,0.029529827,-0.009021567,0.13483547,0.035435885,-0.10254166,0.015250514,-0.2829099,-0.032752763,0.15324983,-0.047394477,-0.074853785,0.04259107,0.24782203,-0.030579925,0.17069864,0.061974112,-0.30386817,0.06484608,0.04618824,0.024469726,0.13064387,0.06397781,-4.8558884,-0.1258333,-0.0

In [434]:
all_gs_results = []
for country in country_list:
    for gs_query, embedding_val in gold_standard_query_embeddings.items():
        query = text("""
        SELECT 
            doc_chunks.doc_id,
            doc_chunks.chunk_index,
            doc_chunks.content,
            documents.country,
            doc_chunks.climate_bert_cluster_labels,
            doc_chunks.w2v_cluster_labels,
            doc_chunks.chunk_metadata,
            (doc_chunks.embedding <=> :embedding) AS cosine_distance
        FROM doc_chunks
        LEFT JOIN documents ON doc_chunks.doc_id = documents.doc_id
        WHERE 
            documents.country = :country
        ORDER BY cosine_distance ASC
        LIMIT 10
        """)

        with engine.connect() as conn:
            result_rows = conn.execute(query, {
                "embedding": embedding_val,
                "country": country
            }).fetchall()

        df_gs_results = pd.DataFrame(result_rows, columns=[
            "doc_id", "chunk_index", "content", "country", 
            "climate_bert_cluster", "w2v_cluster", "chunk_metadata", "cosine_distance"
        ])
        df_gs_results["gs_query"] = gs_query
        all_gs_results.append(df_gs_results)
        df_gs_results["cosine_similarity"] = 1 - df_gs_results["cosine_distance"]

# Combine all into one DataFrame
gs_queries_df = pd.concat(all_gs_results, ignore_index=True)

/var/folders/v6/xrbmfj054b7cb1_bvtvf02nc0000gn/T/ipykernel_65382/1674531613.py:37: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  gs_queries_df = pd.concat(all_gs_results, ignore_index=True)


In [435]:
gs_queries_df['country'].unique()

array(['Bangladesh', 'Brunei Darussalam', 'Cameroon', 'Viet Nam',
       'Uganda', 'Montenegro', 'Jordan', 'Syrian Arab Republic',
       'Dominican Republic', 'Cambodia', 'Papua New Guinea', 'Singapore',
       'San Marino', 'Sri Lanka', 'Uzbekistan', 'Finland',
       'United Kingdom of Great Britain and Northern Ireland', 'Albania',
       'Grenada', 'Ukraine', 'Saudi Arabia', 'Sao Tome and Principe',
       'Cuba', 'Kyrgyzstan', 'Algeria', 'Maldives', 'Djibouti', 'Nauru',
       'Senegal', 'Malaysia', 'Ghana', 'Iceland', 'Zambia', 'Kuwait',
       'Sierra Leone', 'Bosnia and Herzegovina', 'Liberia', 'Philippines',
       'Benin', 'Tuvalu', 'State of Palestine', 'Cabo Verde', 'Rwanda',
       'Tajikistan', 'Zimbabwe', "Lao People's Democratic Republic",
       'China', 'Saint Lucia', 'Armenia', 'Belarus', 'Gabon', 'Australia',
       'Dominica', 'Serbia', 'Angola', 'Vanuatu', 'United Arab Emirates',
       'Georgia', 'Malawi', 'Burundi', 'Democratic Republic of the Congo',
       'B

These already look strong, I have viewed the full content chunks below to ensure we are getting the ones we want

In [436]:
for i in range(0, 1500, 50):
    print(gs_queries_df['content'].iloc[i])

in household and commercial buildings (achieve 19% and 25% reduction in emission respectively)
in household and commercial buildings (achieve 19% and 25% reduction in emission respectively)
+0,6° C en 2025 à +3,6° C en 2100.
Full implementation of the measure will contribute to a GHG emission reduction of approximately 3.37 MtCO2e by 2030.
NDC scenario 2030 19%
 The projected emissions under BAU scenario is estimated to 43,989.0 Gg CO2 eq in year 2030.
GHG emission reduction is estimated at 0.002 MtCO2e/year for 3,500 new electric machines.
Enhancing levels of renewables in the energy mix – through increasing from 30% (2015) to 78% the level of installed capacity for on grid electricity generation that is produced by renewables9.
Not applicable. Singapore’s 2nd NDC is an economy-wide absolute GHG emissions reduction target.
BAU emissions scenario and potential GHG emission reduction pathways for some mitigation
− reduce pollutant emissions by 10% (from 2.5 million to 2.2 million tons)

__These look very promising and already an improvement over the queries we have been running__ <BR>

There are of course some duplicates in the chunks generated by the different queries so we need to filter these out first - doc_id plus chunk_index creates a unique filter for each content block so I will screen based on that


#### Threshold Intrdouction
The only concern at this stage is that some of the chunks seems to be speaking about other targets related to NDC such as electricity, methane etc.. Aside from this there are of course obvious redundant chunks. If my similarity seearch process is robust then, by implementing a threshold filter, I should be able to isolate mostly those content chunks which are (A) specific to emissions and, (B) are not redundant unstructured pieces of text <BR>

I will just do this on my pandas dataframe to avoid having to run the PGVector search and retrieval again

In [437]:
distant_chunks_mask = gs_queries_df['cosine_distance'] > 0.05
print(f'Number of distance chunks (<0.95 cosine similarity distance away:{len(gs_queries_df[distant_chunks_mask])}')
gs_queries_df[distant_chunks_mask] 

Number of distance chunks (<0.95 cosine similarity distance away:58


,doc_id,chunk_index,content,country,climate_bert_cluster,w2v_cluster,chunk_metadata,cosine_distance,gs_query,cosine_similarity
100,cameroon_french_20220601,898,"+0,6° C en 2025 à +3,6° C en 2100.",Cameroon,Narrative - English,Fragmented Phrases & Minor Details,"{'links': [], 'filename': 'cameroon_french_202...",0.057391,reduce greenhouse gas emissions % below levels...,0.942609
102,cameroon_french_20220601,898,"+0,6° C en 2025 à +3,6° C en 2100.",Cameroon,Narrative - English,Fragmented Phrases & Minor Details,"{'links': [], 'filename': 'cameroon_french_202...",0.053146,% reduction of GHG emissions by 2030. Which is...,0.946854
106,cameroon_french_20220601,898,"+0,6° C en 2025 à +3,6° C en 2100.",Cameroon,Narrative - English,Fragmented Phrases & Minor Details,"{'links': [], 'filename': 'cameroon_french_202...",0.060834,NDC target is to reduce greenhouse gas emissio...,0.939166
108,cameroon_french_20220601,898,"+0,6° C en 2025 à +3,6° C en 2100.",Cameroon,Narrative - English,Fragmented Phrases & Minor Details,"{'links': [], 'filename': 'cameroon_french_202...",0.061507,aims to reduce its greenhouse gas emissions by...,0.938493
246,jordan_english_20220601,154,Sectors (%) 100% 81% 81%,Jordan,"Tabular, Numeric",Emission Reduction Targets & Commitments,"{'links': [], 'filename': 'jordan_english_2022...",0.052947,The Estimated Mitigation Target is X% by 2030,0.947053
277,dominican_republic_spanish_20220601,2463,Planificación (2023-2025) Ejecución (2025-2030),Dominican Republic,Narrative - English,Formal Climate Pledges – Spanish,"{'links': [], 'filename': 'dominican_republic_...",0.051052,The Estimated Mitigation Target is X% by 2030,0.948948
278,dominican_republic_spanish_20220601,2262,Planificación (2021-2023) Ejecución (2023-2025),Dominican Republic,Narrative - English,Formal Climate Pledges – Spanish,"{'links': [], 'filename': 'dominican_republic_...",0.053388,The Estimated Mitigation Target is X% by 2030,0.946612
351,papua_new_guinea_english_20220601,28,This will be achieved by:,Papua New Guinea,Narrative - English,Emission Reduction Targets & Commitments,"{'links': [], 'filename': 'papua_new_guinea_en...",0.051822,The Estimated Mitigation Target is X% by 2030,0.948178
391,singapore_english_20250201,27,1 January 2031 to 31 December 2035.,Singapore,Narrative - English,Fragmented Phrases & Minor Details,"{'links': [], 'filename': 'singapore_english_2...",0.052205,The Estimated Mitigation Target is X% by 2030,0.947795
434,sri_lanka_english_20220601,404,by improving animal comfort and welfare (40% i...,Sri Lanka,Narrative - English,Legal & Institutional Disclaimers,"{'links': [], 'filename': 'sri_lanka_english_2...",0.052861,reduce greenhouse gas emissions % below levels...,0.947139


The above filters out 58 chunks - they all look redundant so I will look to see if implementing an even tighter threshold helps with screening or if it skims some relevant chunks

In [438]:
distant_96_chunks_mask = gs_queries_df['cosine_distance'] > 0.04
print(f'Number of distance chunks (<0.95 cosine similarity distance away:{len(gs_queries_df[distant_96_chunks_mask])}')

for i in range(0, len(gs_queries_df[distant_96_chunks_mask]), 10):
    print(gs_queries_df[distant_96_chunks_mask]['content'].iloc[i])

Number of distance chunks (<0.95 cosine similarity distance away:299
In unconditional scenario, GHG emissions would be reduced by 27.56 Mt CO2e (6.73%) below BAU in 2030 and in conditional scenario, GHG emissions would be reduced by 89.47 Mt CO2e (21.85%) below BAU in 2030 in the respective sectors.
+0,6° C en 2025 à +3,6° C en 2100.
Planificación: (2020-2022) Ejecución (2022-2030)
Base year: 2005
projected high growth in the near future;
Ukraine ranks 56th in the world by the size of nominal GDP in absolute terms, and 119th by size of GDP per capita.
The adaptation projects currently being implemented account for a budget of nearly €100 million. However, that sum represents just 12% of the total amount that will need to be invested in adaptation under the 2°C Scenario, and a mere 7.5% in the case of the 4°C Scenario.
d. Target relative to the reference indicator, expressed numerically, for example, in percentage or amount of reduction.
The Philippines is a low-middle income developing

Majority of these look to be irrelevant however, some chunks are relevant - I will therefore adopt a threshold slightly below this at 0.958

In [439]:
irrelevant_chunks_mask = gs_queries_df['cosine_distance'] > 0.042
print(gs_queries_df[irrelevant_chunks_mask])
print(f'We will be removing: {len(gs_queries_df[irrelevant_chunks_mask])} chunks')

irrelevant_chunks_mask = gs_queries_df['cosine_distance'] > 0.042
gs_queries_df_partial_clean = gs_queries_df[~irrelevant_chunks_mask].reset_index(drop=True)


                                    doc_id chunk_index  \
19             bangladesh_english_20220601         886   
81      brunei_darussalam_english_20220601          25   
82      brunei_darussalam_english_20220601           7   
98                cameroon_french_20220601         898   
100               cameroon_french_20220601         898   
...                                    ...         ...   
4958  antigua_and_barbuda_english_20220601         367   
4959  antigua_and_barbuda_english_20220601         226   
4991              andorra_spanish_20250201         140   
4998              andorra_spanish_20250201         140   
5001              andorra_spanish_20250201         140   

                                                content              country  \
19    The updated unconditional and conditional cont...           Bangladesh   
81    1 January 2021 – 31 December 2030 (10-year per...    Brunei Darussalam   
82                                      Base year: 2015    Brun

In [440]:
gs_queries_df_clean= gs_queries_df_partial_clean.drop_duplicates(subset=["doc_id", "chunk_index"], keep="first").reset_index(drop=True)

#This drops the number of rows to 1773, this is very good when we consider that we want to pass as few chunks/tokens into the LLM to keep within the limit and also to not overload with unnecessary information
gs_queries_df_clean

,doc_id,chunk_index,content,country,climate_bert_cluster,w2v_cluster,chunk_metadata,cosine_distance,gs_query,cosine_similarity
0,bangladesh_english_20220601,517,in household and commercial buildings (achieve...,Bangladesh,Narrative - English,Emission Reduction Targets & Commitments,"{'links': [], 'filename': 'bangladesh_english_...",0.021411,2030 target is a % reduction in ghgs below lev...,0.978589
1,bangladesh_english_20220601,377,in household and commercial buildings (achieve...,Bangladesh,Narrative - English,Emission Reduction Targets & Commitments,"{'links': [], 'filename': 'bangladesh_english_...",0.021864,2030 target is a % reduction in ghgs below lev...,0.978136
2,bangladesh_english_20220601,745,Total GHG emission will be 409.4 MtCO2e in 203...,Bangladesh,Narrative - English,Emission Reduction Targets & Commitments,"{'links': [], 'filename': 'bangladesh_english_...",0.023136,2030 target is a % reduction in ghgs below lev...,0.976864
3,bangladesh_english_20220601,10,Bangladesh’s INDC proposed for 12 million tons...,Bangladesh,Narrative - English,Emission Reduction Targets & Commitments,"{'links': [], 'filename': 'bangladesh_english_...",0.028163,2030 target is a % reduction in ghgs below lev...,0.971837
4,bangladesh_english_20220601,740,Total GHG emission accounts for 169.05 MtCO2e ...,Bangladesh,Narrative - English,Emission Reduction Targets & Commitments,"{'links': [], 'filename': 'bangladesh_english_...",0.029066,2030 target is a % reduction in ghgs below lev...,0.970934
...,...,...,...,...,...,...,...,...,...,...
1669,somalia_english_20220601,198,"Estimated investment required (in USD, 2021-20...",Somalia,Narrative - English,Population & Infrastructure Projections,"{'links': [], 'filename': 'somalia_english_202...",0.028079,The Estimated Mitigation Target is X% by 2030,0.971921
1670,andorra_spanish_20250201,135,"in net greenhouse gas emissions by 2035, compa...",Andorra,Narrative - English,Emission Reduction Targets & Commitments,"{'links': [], 'filename': 'andorra_spanish_202...",0.021906,2030 target is a % reduction in ghgs below lev...,0.978094
1671,andorra_spanish_20250201,136,"2005 levels, with the ambition of achieving ca...",Andorra,Narrative - English,Emission Reduction Targets & Commitments,"{'links': [], 'filename': 'andorra_spanish_202...",0.024723,2030 target is a % reduction in ghgs below lev...,0.975277
1672,andorra_spanish_20250201,140,"target based on a ""Business as Usual"" (BAU) sc...",Andorra,Narrative - English,Emission Reduction Targets & Commitments,"{'links': [], 'filename': 'andorra_spanish_202...",0.036083,2030 target is a % reduction in ghgs below lev...,0.963917


In [441]:
#Quick calculation to see if we are on track in terms of the number of tokens we are going to be passing in to the LLM
len(gs_queries_df_clean) / len(gs_queries_df_clean['country'].unique())

#13 chunks per country on average - assuming with the other retrieved chunks we end passing in 20 chunks to the LLM, this should be fine given the number of tokens I have

12.976744186046512

In [362]:
#Manipulate the gs dataframe so that columns match the final_df -> ready for merging
final_gs_queries = gs_queries_df_clean.copy()
final_gs_queries.drop(columns='gs_query', inplace=True)

if list(final_gs_queries.columns) == list(final_df.columns):
    print('Columns are the same, ready for merging')

Columns are the same, ready for merging


In [309]:
complete_embeddings_df = pd.concat([final_df, final_gs_queries], ignore_index=True)

In [363]:
complete_embeddings_df = complete_embeddings_df.drop_duplicates(subset=["doc_id", "chunk_index"]).reset_index(drop=True)
complete_embeddings_df

,doc_id,chunk_index,content,country,climate_bert_cluster,w2v_cluster,chunk_metadata,cosine_distance,cosine_similarity
0,bangladesh_english_20220601,745,Total GHG emission will be 409.4 MtCO2e in 203...,Bangladesh,Narrative - English,Emission Reduction Targets & Commitments,"{'links': [], 'filename': 'bangladesh_english_...",0.024598,0.975402
1,bangladesh_english_20220601,10,Bangladesh’s INDC proposed for 12 million tons...,Bangladesh,Narrative - English,Emission Reduction Targets & Commitments,"{'links': [], 'filename': 'bangladesh_english_...",0.036378,0.963622
2,bangladesh_english_20220601,92,Total GHG emission is found to be increased fr...,Bangladesh,Narrative - English,Emission Reduction Targets & Commitments,"{'links': [], 'filename': 'bangladesh_english_...",0.040419,0.959581
3,bangladesh_english_20220601,588,"Under this comprehensive plan, the government ...",Bangladesh,Narrative - English,Emission Reduction Targets & Commitments,"{'links': [], 'filename': 'bangladesh_english_...",0.042267,0.957733
4,bangladesh_english_20220601,886,The updated unconditional and conditional cont...,Bangladesh,Narrative - English,Emission Reduction Targets & Commitments,"{'links': [], 'filename': 'bangladesh_english_...",0.044091,0.955909
...,...,...,...,...,...,...,...,...,...
1726,somalia_english_20220601,198,"Estimated investment required (in USD, 2021-20...",Somalia,Narrative - English,Population & Infrastructure Projections,"{'links': [], 'filename': 'somalia_english_202...",0.028079,0.971921
1727,andorra_spanish_20250201,135,"in net greenhouse gas emissions by 2035, compa...",Andorra,Narrative - English,Emission Reduction Targets & Commitments,"{'links': [], 'filename': 'andorra_spanish_202...",0.021906,0.978094
1728,andorra_spanish_20250201,136,"2005 levels, with the ambition of achieving ca...",Andorra,Narrative - English,Emission Reduction Targets & Commitments,"{'links': [], 'filename': 'andorra_spanish_202...",0.024723,0.975277
1729,andorra_spanish_20250201,140,"target based on a ""Business as Usual"" (BAU) sc...",Andorra,Narrative - English,Emission Reduction Targets & Commitments,"{'links': [], 'filename': 'andorra_spanish_202...",0.036083,0.963917


In [314]:
# The final number of chunks is 13.4 per country - some will of course have more and some less 
len(complete_embeddings_df) / len(complete_embeddings_df['country'].unique())

13.418604651162791

In [322]:
complete_embeddings_df

,doc_id,chunk_index,content,country,climate_bert_cluster,w2v_cluster,chunk_metadata,cosine_distance,cosine_similarity
0,bangladesh_english_20220601,745,Total GHG emission will be 409.4 MtCO2e in 203...,Bangladesh,Narrative - English,Emission Reduction Targets & Commitments,"{'links': [], 'filename': 'bangladesh_english_...",0.024598,0.975402
1,bangladesh_english_20220601,10,Bangladesh’s INDC proposed for 12 million tons...,Bangladesh,Narrative - English,Emission Reduction Targets & Commitments,"{'links': [], 'filename': 'bangladesh_english_...",0.036378,0.963622
2,bangladesh_english_20220601,92,Total GHG emission is found to be increased fr...,Bangladesh,Narrative - English,Emission Reduction Targets & Commitments,"{'links': [], 'filename': 'bangladesh_english_...",0.040419,0.959581
3,bangladesh_english_20220601,588,"Under this comprehensive plan, the government ...",Bangladesh,Narrative - English,Emission Reduction Targets & Commitments,"{'links': [], 'filename': 'bangladesh_english_...",0.042267,0.957733
4,bangladesh_english_20220601,886,The updated unconditional and conditional cont...,Bangladesh,Narrative - English,Emission Reduction Targets & Commitments,"{'links': [], 'filename': 'bangladesh_english_...",0.044091,0.955909
...,...,...,...,...,...,...,...,...,...
1726,somalia_english_20220601,198,"Estimated investment required (in USD, 2021-20...",Somalia,Narrative - English,Population & Infrastructure Projections,"{'links': [], 'filename': 'somalia_english_202...",0.028079,0.971921
1727,andorra_spanish_20250201,135,"in net greenhouse gas emissions by 2035, compa...",Andorra,Narrative - English,Emission Reduction Targets & Commitments,"{'links': [], 'filename': 'andorra_spanish_202...",0.021906,0.978094
1728,andorra_spanish_20250201,136,"2005 levels, with the ambition of achieving ca...",Andorra,Narrative - English,Emission Reduction Targets & Commitments,"{'links': [], 'filename': 'andorra_spanish_202...",0.024723,0.975277
1729,andorra_spanish_20250201,140,"target based on a ""Business as Usual"" (BAU) sc...",Andorra,Narrative - English,Emission Reduction Targets & Commitments,"{'links': [], 'filename': 'andorra_spanish_202...",0.036083,0.963917


In [323]:
countries = complete_embeddings_df['country'].unique()
countries

array(['Bangladesh', 'Brunei Darussalam', 'Cameroon', 'Viet Nam',
       'Uganda', 'Montenegro', 'Jordan', 'Cambodia', 'Papua New Guinea',
       'Singapore', 'Sri Lanka', 'Uzbekistan', 'Finland',
       'United Kingdom of Great Britain and Northern Ireland', 'Albania',
       'Grenada', 'Ukraine', 'Saudi Arabia', 'Sao Tome and Principe',
       'Cuba', 'Kyrgyzstan', 'Algeria', 'Maldives', 'Djibouti',
       'Malaysia', 'Ghana', 'Iceland', 'Zambia', 'Bosnia and Herzegovina',
       'Liberia', 'Philippines', 'Tuvalu', 'State of Palestine',
       'Cabo Verde', 'Rwanda', 'Tajikistan', 'Zimbabwe',
       "Lao People's Democratic Republic", 'China', 'Saint Lucia',
       'Armenia', 'Belarus', 'Australia', 'Dominica', 'Serbia', 'Angola',
       'Vanuatu', 'United Arab Emirates', 'Georgia', 'Bhutan',
       'Republic of Korea', 'Solomon Islands', 'Thailand', 'Togo',
       'Burkina Faso', 'Oman', 'Micronesia (Federated States of)', 'Fiji',
       'Holy See', 'Eritrea', 'Canada', 'Barbados',


In [328]:
complete_embeddings_df

,doc_id,chunk_index,content,country,climate_bert_cluster,w2v_cluster,chunk_metadata,cosine_distance,cosine_similarity
0,bangladesh_english_20220601,745,Total GHG emission will be 409.4 MtCO2e in 203...,Bangladesh,Narrative - English,Emission Reduction Targets & Commitments,"{'links': [], 'filename': 'bangladesh_english_...",0.024598,0.975402
1,bangladesh_english_20220601,10,Bangladesh’s INDC proposed for 12 million tons...,Bangladesh,Narrative - English,Emission Reduction Targets & Commitments,"{'links': [], 'filename': 'bangladesh_english_...",0.036378,0.963622
2,bangladesh_english_20220601,92,Total GHG emission is found to be increased fr...,Bangladesh,Narrative - English,Emission Reduction Targets & Commitments,"{'links': [], 'filename': 'bangladesh_english_...",0.040419,0.959581
3,bangladesh_english_20220601,588,"Under this comprehensive plan, the government ...",Bangladesh,Narrative - English,Emission Reduction Targets & Commitments,"{'links': [], 'filename': 'bangladesh_english_...",0.042267,0.957733
4,bangladesh_english_20220601,886,The updated unconditional and conditional cont...,Bangladesh,Narrative - English,Emission Reduction Targets & Commitments,"{'links': [], 'filename': 'bangladesh_english_...",0.044091,0.955909
...,...,...,...,...,...,...,...,...,...
1726,somalia_english_20220601,198,"Estimated investment required (in USD, 2021-20...",Somalia,Narrative - English,Population & Infrastructure Projections,"{'links': [], 'filename': 'somalia_english_202...",0.028079,0.971921
1727,andorra_spanish_20250201,135,"in net greenhouse gas emissions by 2035, compa...",Andorra,Narrative - English,Emission Reduction Targets & Commitments,"{'links': [], 'filename': 'andorra_spanish_202...",0.021906,0.978094
1728,andorra_spanish_20250201,136,"2005 levels, with the ambition of achieving ca...",Andorra,Narrative - English,Emission Reduction Targets & Commitments,"{'links': [], 'filename': 'andorra_spanish_202...",0.024723,0.975277
1729,andorra_spanish_20250201,140,"target based on a ""Business as Usual"" (BAU) sc...",Andorra,Narrative - English,Emission Reduction Targets & Commitments,"{'links': [], 'filename': 'andorra_spanish_202...",0.036083,0.963917


### LLM API Interaction - The Generative Portion of My RAG Pipeline

The final step in my pipeline is to 'generate' responses to the original question <BR>
We have completed the retrieval stage as is evident in the above dataframe - now is a question of generating the optimal response using LLM responses based on the fed in chunks we have identified

In [365]:
## CAN IGNORE
# Practice cell for formatting context to pass to the LLM - 
def format_context(row):
    return f"Doc ID: {row['doc_id']}, Chunk ID: {row['chunk_index']}, Chunk Content: '{row['content']}'"

for country in countries:
    formatted_chunks = ''
    relevant_df = complete_embeddings_df[complete_embeddings_df['country'] == country]

    for i in range(len(relevant_df)):
        row = relevant_df.iloc[i]
        formatted_chunk = format_context(row)
        formatted_chunks += formatted_chunk + '\n'
    
    formatted_chunks 

print(formatted_chunks)

Doc ID: andorra_spanish_20250201, Chunk ID: 135, Chunk Content: 'in net greenhouse gas emissions by 2035, compared to'
Doc ID: andorra_spanish_20250201, Chunk ID: 136, Chunk Content: '2005 levels, with the ambition of achieving carbon neutrality by'
Doc ID: andorra_spanish_20250201, Chunk ID: 140, Chunk Content: 'target based on a "Business as Usual" (BAU) scenario to an'
Doc ID: andorra_spanish_20250201, Chunk ID: 134, Chunk Content: 'at COP28. This document establishes a 63% reduction target'



Below is my process for generating the queries to feed to the LLM through our API key. I ended up experimenting with 3 different system prompts to give to the LLM. Whilst results are removed, I tested each one and ended up using system_prompt_c for the final workflow

In [398]:
country_message_directory = {}
system_prompt_a = (
    "You are a climate policy expert. Answer the question using ONLY the provided document chunks below. "
    "Your answer must clearly and concisely state the country's 2030 emissions reduction target. "
    "Do not make assumptions or use information not present in the chunks. "
    "Cite sources using [Doc ID: <id>, Chunk ID: <id>]. "
    "Your answer should be specific and clearly state the emissions reduction target by 2030, be concise where possible."
    " If the answer is not clearly stated, respond with: 'The target is not clearly specified in the provided chunks.'"
)

system_prompt_b = (
    "You are a climate policy analyst. Based only on the following document chunks, extract the official 2030 emissions reduction target for the country. "
    "If both unconditional and conditional targets exist, return both clearly. "
    "Include % reduction, baseline year or scenario (e.g. BAU, 1990), and any numerical estimates (e.g. MtCO2e). "
    "Always cite the relevant chunk in the format [Doc ID: <id>, Chunk ID: <id>]. "
    "If a target is not stated, write: 'Target not found in provided chunks.'"
)

system_prompt_c = (
    "You are a climate policy expert. Based only on the provided document chunks, extract the country’s 2030 emissions reduction target. "
    "If both unconditional and conditional targets exist, return both clearly. Include % reductions, baseline year/scenario (e.g. BAU), and any quantitative values (e.g. MtCO2e). "
    "Be concise. Always cite the relevant chunks using [Doc ID: <id>, Chunk ID: <id>]. "
    "If a target is not found, respond: 'The target is not clearly specified in the provided chunks.'"
)

def format_context(row):
    return f"[Doc ID: {row['doc_id']}, Chunk ID: {row['chunk_index']}] \n '{row['content']}'"


# Here I am constructing the dictionary of prompts that I will give to the API indexing by country - note that for each coutry I restrict the number of chunks to 10
for country in countries:
    formatted_chunks = ''
    relevant_df = complete_embeddings_df[complete_embeddings_df['country'] == country]
    query_text = f"What is the official 2030 emissions reduction target for {country}, based on the provided document chunks?"
    
    for i in range(len(relevant_df)):
        if i == 10:
            break
        row = relevant_df.iloc[i]
        formatted_chunk = format_context(row)
        formatted_chunks += formatted_chunk + '\n'
            
    messages = [
        {"role": "system", "content": system_prompt_c},
        {"role": "user", "content": f"Context:\n{formatted_chunks.strip()}\n\nQuestion: {query_text}\n\nAnswer:"}]
    
    for row in messages:
        for key, value in row.items():
            print(f"{key}: {value}")
    
    country_message_directory[country] = messages
    
country_message_directory

role: system
content: You are a climate policy expert. Based only on the provided document chunks, extract the country’s 2030 emissions reduction target. If both unconditional and conditional targets exist, return both clearly. Include % reductions, baseline year/scenario (e.g. BAU), and any quantitative values (e.g. MtCO2e). Be concise. Always cite the relevant chunks using [Doc ID: <id>, Chunk ID: <id>]. If a target is not found, respond: 'The target is not clearly specified in the provided chunks.'
role: user
content: Context:
[Doc ID: bangladesh_english_20220601, Chunk ID: 745] 
 'Total GHG emission will be 409.4 MtCO2e in 2030 under BAU scenario. The contributions of the sectors are;'
[Doc ID: bangladesh_english_20220601, Chunk ID: 10] 
 'Bangladesh’s INDC proposed for 12 million tons (5%) unconditional reduction in GHG emission from Business as Usual (BAU) scenario by 2030 and a further 24 million tons (10%) conditional reduction in GHG emission with support from the internationa

{'Bangladesh': [{'role': 'system',
   'content': "You are a climate policy expert. Based only on the provided document chunks, extract the country’s 2030 emissions reduction target. If both unconditional and conditional targets exist, return both clearly. Include % reductions, baseline year/scenario (e.g. BAU), and any quantitative values (e.g. MtCO2e). Be concise. Always cite the relevant chunks using [Doc ID: <id>, Chunk ID: <id>]. If a target is not found, respond: 'The target is not clearly specified in the provided chunks.'"},
  {'role': 'user',
   'content': "Context:\n[Doc ID: bangladesh_english_20220601, Chunk ID: 745] \n 'Total GHG emission will be 409.4 MtCO2e in 2030 under BAU scenario. The contributions of the sectors are;'\n[Doc ID: bangladesh_english_20220601, Chunk ID: 10] \n 'Bangladesh’s INDC proposed for 12 million tons (5%) unconditional reduction in GHG emission from Business as Usual (BAU) scenario by 2030 and a further 24 million tons (10%) conditional reduction i

In [399]:
from dotenv import load_dotenv
from openai import OpenAI
import os
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("roberta-base")
load_dotenv()  # Load from .env

api_key = os.getenv("NEBIUS_API_KEY")
llm_model = os.getenv("LLM_MODEL")

client = OpenAI(
    base_url="https://api.studio.nebius.com/v1/",
    api_key=api_key
)

llm_responses = {}
for country, message in country_message_directory.items():
    print(f"Processing country: {country}")
    flattened_text = "".join([m["content"] for m in message])
    tokenized = tokenizer(flattened_text, return_tensors="pt", truncation = True)
    num_tokens = len(tokenized["input_ids"][0])
    
    print(f"[{country}] Token count: {num_tokens}")
    response = client.chat.completions.create(
        model=llm_model,
        max_tokens=1024,
        temperature=0.6,
        top_p=0.9,
        extra_body={
            "top_k": 50
        },
        messages=message
    )
    print(f"API response returned for: {country}")

    llm_responses[country] = response.choices[0].message.content

Processing country: Bangladesh
[Bangladesh] Token count: 512
API response returned for: Bangladesh
Processing country: Brunei Darussalam
[Brunei Darussalam] Token count: 512
API response returned for: Brunei Darussalam
Processing country: Cameroon
[Cameroon] Token count: 221
API response returned for: Cameroon
Processing country: Viet Nam
[Viet Nam] Token count: 469
API response returned for: Viet Nam
Processing country: Uganda
[Uganda] Token count: 512
API response returned for: Uganda
Processing country: Montenegro
[Montenegro] Token count: 458
API response returned for: Montenegro
Processing country: Jordan
[Jordan] Token count: 512
API response returned for: Jordan
Processing country: Cambodia
[Cambodia] Token count: 512
API response returned for: Cambodia
Processing country: Papua New Guinea
[Papua New Guinea] Token count: 441
API response returned for: Papua New Guinea
Processing country: Singapore
[Singapore] Token count: 512
API response returned for: Singapore
Processing count

In [401]:
llm_responses

{'Bangladesh': 'The 2030 emissions reduction target for Bangladesh is as follows:\n\n* Unconditional target: 27.56 MtCO2e (6.73%) reduction below BAU in 2030 [Doc ID: bangladesh_english_20220601, Chunk ID: 755]\n* Conditional target: 89.47 MtCO2e (21.85%) reduction below BAU in 2030 [Doc ID: bangladesh_english_20220601, Chunk ID: 755]\n\nNote that the BAU scenario is 409.4 MtCO2e in 2030 [Doc ID: bangladesh_english_20220601, Chunk ID: 745].',
 'Brunei Darussalam': 'The official 2030 emissions reduction target for Brunei Darussalam is a reduction in GHG emissions by 20% relative to Business-As-Usual (BAU) levels by 2030 [Doc ID: brunei_darussalam_english_20220601, Chunk ID: 16 and 5].',
 'Cameroon': 'The target is not clearly specified in the provided chunks.',
 'Viet Nam': "Viet Nam's 2030 emissions reduction targets are:\n\n- Unconditional contribution: 15.8% reduction compared to BAU [Doc ID: viet_nam_english_20221101, Chunk ID: 74]\n- Conditional contribution: 43.5% reduction compar

In [410]:
failed_countries = []
successful_countries = []
for country, response in llm_responses.items():
    if response == 'The target is not clearly specified in the provided chunks.':
        failed_countries.append(country)
    else:
        successful_countries.append(country)


print(f'Out of a total of {len(failed_countries + successful_countries)} countries, we managed to obtain answers for {len(successful_countries) } or {round(len(successful_countries) / len(failed_countries + successful_countries) * 100)}% of them')

Out of a total: 129 countries, we managed to obtain answers for 78 or 60% of them


### Constructing our final data frame to store the responses from the LLM

In [418]:
import pandas as pd
import re
llm_df = pd.DataFrame([{"country": country, "llm_response": response} for country, response in llm_responses.items()])

def extract_citations(text):
    pattern = r"\[Doc ID: (.*?), Chunk ID: (\d+)\]"
    return re.findall(pattern, text)

llm_df["citations"] = llm_df["llm_response"].apply(extract_citations)
exploded = llm_df.explode("citations").dropna(subset=["citations"])

exploded[["doc_id", "chunk_index"]] = pd.DataFrame(exploded["citations"].tolist(), index=exploded.index)
exploded["chunk_index"] = exploded["chunk_index"].astype(int)

merged_df = pd.merge(exploded, complete_embeddings_df,on=["doc_id", "chunk_index"],how="left")

merged_df = merged_df.drop(columns=["citations"])


### Second Retrieval Loop

Because our pipeline failed to retrieve information for almost 40% of countries, I wanted to explore how we might obtain more answers. I noticed that the countries where we didnt obtain a generated answer were those ones which had less chunks fed to the LLM. This possibly was because we imposed too strict of thresholds earlier. I now wanted to re-run our retrieval process for those countries we couldn't obtain answers for re-running the similarity search this time with no thresholds, and higher number of returned chunks

In [466]:
non_retrieved_countries = list(failed_countries)

In [467]:
all_non_retrieved_country_results = []

for country in non_retrieved_countries:
    for gs_query, embedding_val in gold_standard_query_embeddings.items():
        query = text("""
        SELECT 
            doc_chunks.doc_id,
            doc_chunks.chunk_index,
            doc_chunks.content,
            documents.country,
            doc_chunks.climate_bert_cluster_labels,
            doc_chunks.w2v_cluster_labels,
            doc_chunks.chunk_metadata,
            (doc_chunks.embedding <=> :embedding) AS cosine_distance
        FROM doc_chunks
        LEFT JOIN documents ON doc_chunks.doc_id = documents.doc_id
        WHERE 
            documents.country = :country
        ORDER BY cosine_distance ASC
        LIMIT 15
        """)

        with engine.connect() as conn:
            result_rows = conn.execute(query, {
                "embedding": embedding_val,
                "country": country
            }).fetchall()

        rem_country_results = pd.DataFrame(result_rows, columns=[
            "doc_id", "chunk_index", "content", "country", 
            "climate_bert_cluster", "w2v_cluster", "chunk_metadata", "cosine_distance"
        ])
        rem_country_results["gs_query"] = gs_query
        rem_country_results["cosine_similarity"] = 1 - rem_country_results["cosine_distance"]
        
        all_non_retrieved_country_results.append(rem_country_results)

# Combine into one DataFrame
rem_countries_df = pd.concat(all_non_retrieved_country_results, ignore_index=True)
rem_countries_df = rem_countries_df.drop_duplicates(subset=["doc_id", "chunk_index"]).reset_index(drop=True)

/var/folders/v6/xrbmfj054b7cb1_bvtvf02nc0000gn/T/ipykernel_65382/1618816941.py:39: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  rem_countries_df = pd.concat(all_non_retrieved_country_results, ignore_index=True)


In [468]:
rem_countries_df

,doc_id,chunk_index,content,country,climate_bert_cluster,w2v_cluster,chunk_metadata,cosine_distance,gs_query,cosine_similarity
0,cameroon_french_20220601,677,"CO2, soit 12 % des émissions dans le scenario ...",Cameroon,Narrative - English,Development & Access Projects – French,"{'links': [], 'filename': 'cameroon_french_202...",0.029144,2030 target is a % reduction in ghgs below lev...,0.970856
1,cameroon_french_20220601,898,"+0,6° C en 2025 à +3,6° C en 2100.",Cameroon,Narrative - English,Fragmented Phrases & Minor Details,"{'links': [], 'filename': 'cameroon_french_202...",0.049589,2030 target is a % reduction in ghgs below lev...,0.950411
2,cameroon_french_20220601,569,"Scénario ""with measures"" 2030",Cameroon,"Headers, Keywords, Multilingual",Emission Reduction Targets & Commitments,"{'links': [], 'filename': 'cameroon_french_202...",0.037173,The Estimated Mitigation Target is X% by 2030,0.962827
3,cameroon_french_20220601,629,électrique en 2035 »,Cameroon,"Headers, Keywords, Multilingual",Development & Access Projects – French,"{'links': [], 'filename': 'cameroon_french_202...",0.048699,The Estimated Mitigation Target is X% by 2030,0.951301
4,papua_new_guinea_english_20220601,27,Energy Industries: Within the Energy Industrie...,Papua New Guinea,Narrative - English,Project Sectors & Technologies,"{'links': [], 'filename': 'papua_new_guinea_en...",0.028881,2030 target is a % reduction in ghgs below lev...,0.971119
...,...,...,...,...,...,...,...,...,...,...
552,republic_of_moldova_english_20220601,476,Extreme weather events: - - - - -,Republic of Moldova,"Tabular, Numeric",Legal & Institutional Disclaimers,"{'links': [], 'filename': 'republic_of_moldova...",0.081684,The Estimated Mitigation Target is X% by 2030,0.918316
553,andorra_spanish_20250201,135,"in net greenhouse gas emissions by 2035, compa...",Andorra,Narrative - English,Emission Reduction Targets & Commitments,"{'links': [], 'filename': 'andorra_spanish_202...",0.021906,2030 target is a % reduction in ghgs below lev...,0.978094
554,andorra_spanish_20250201,136,"2005 levels, with the ambition of achieving ca...",Andorra,Narrative - English,Emission Reduction Targets & Commitments,"{'links': [], 'filename': 'andorra_spanish_202...",0.024723,2030 target is a % reduction in ghgs below lev...,0.975277
555,andorra_spanish_20250201,140,"target based on a ""Business as Usual"" (BAU) sc...",Andorra,Narrative - English,Emission Reduction Targets & Commitments,"{'links': [], 'filename': 'andorra_spanish_202...",0.036083,2030 target is a % reduction in ghgs below lev...,0.963917


Here I can already see some of the returned chunks are of low quality. This is where the clustering is useful, because we can now remove those obviously incorrect chunks by filtering out the poor clusters


In [ ]:
rem_countries_df = rem_countries_df[rem_countries_df['w2v_cluster'] != 'Fragmented Phrases & Minor Details']
rem_countries_df

,doc_id,chunk_index,content,country,climate_bert_cluster,w2v_cluster,chunk_metadata,cosine_distance,gs_query,cosine_similarity
0,nepal_english_20220601,23,aims to achieve net-zero greenhouse gas emissi...,Nepal,Narrative - English,Emission Reduction Targets & Commitments,"{'links': [], 'filename': 'nepal_english_20220...",0.025414,2030 target is a % reduction in ghgs below lev...,0.974586
1,nepal_english_20220601,141,"By 2030, including measures in policies to off...",Nepal,Narrative - English,Legal & Institutional Disclaimers,"{'links': [], 'filename': 'nepal_english_20220...",0.026306,2030 target is a % reduction in ghgs below lev...,0.973694
2,nepal_english_20220601,311,This NDC broadens the ambition of the 2016 NDC...,Nepal,Narrative - English,Emission Reduction Targets & Commitments,"{'links': [], 'filename': 'nepal_english_20220...",0.028036,2030 target is a % reduction in ghgs below lev...,0.971964
3,nepal_english_20220601,22,long-term low greenhouse gas emission developm...,Nepal,Narrative - English,Mitigation & Adaptation Strategies – English,"{'links': [], 'filename': 'nepal_english_20220...",0.028532,2030 target is a % reduction in ghgs below lev...,0.971468
4,nepal_english_20220601,140,"By 2030, ensure at least five tourist destinat...",Nepal,Narrative - English,Legal & Institutional Disclaimers,"{'links': [], 'filename': 'nepal_english_20220...",0.028684,2030 target is a % reduction in ghgs below lev...,0.971316
...,...,...,...,...,...,...,...,...,...,...
1079,micronesia_(federated_states_of)_english_20221001,274,"Conditional By 2030, increase electricity gene...",Micronesia (Federated States of),Narrative - English,Project Sectors & Technologies,"{'links': [], 'filename': 'micronesia_(federat...",0.029710,2030 target is a % reduction in ghgs below lev...,0.970290
1080,micronesia_(federated_states_of)_english_20221001,67,"By 2030, increase electricity generation from ...",Micronesia (Federated States of),Narrative - English,Project Sectors & Technologies,"{'links': [], 'filename': 'micronesia_(federat...",0.030883,2030 target is a % reduction in ghgs below lev...,0.969117
1081,micronesia_(federated_states_of)_english_20221001,279,Carbon dioxide emissions reductions • Reduced ...,Micronesia (Federated States of),Narrative - English,Emission Reduction Targets & Commitments,"{'links': [], 'filename': 'micronesia_(federat...",0.036580,2030 target is a % reduction in ghgs below lev...,0.963420
1082,micronesia_(federated_states_of)_english_20221001,96,Mitigation Co-benefits • Reduced emissions of ...,Micronesia (Federated States of),Narrative - English,Emission Reduction Targets & Commitments,"{'links': [], 'filename': 'micronesia_(federat...",0.042539,2030 target is a % reduction in ghgs below lev...,0.957461


In [469]:
second_loop_country_message_directory = {}

for country in non_retrieved_countries:
    formatted_chunks = ''
    relevant_df = rem_countries_df[rem_countries_df['country'] == country]
    query_text = f"What is the official 2030 emissions reduction target for {country}, based on the provided document chunks?"
    
    for i in range(len(relevant_df)):
        row = relevant_df.iloc[i]
        formatted_chunk = format_context(row)
        formatted_chunks += formatted_chunk + '\n'
            
    messages = [
        {"role": "system", "content": system_prompt_c},
        {"role": "user", "content": f"Context:\n{formatted_chunks.strip()}\n\nQuestion: {query_text}\n\nAnswer:"}]
    
    for row in messages:
        for key, value in row.items():
            print(f"{key}: {value}")
    
    second_loop_country_message_directory[country] = messages
second_loop_country_message_directory

role: system
content: You are a climate policy expert. Based only on the provided document chunks, extract the country’s 2030 emissions reduction target. If both unconditional and conditional targets exist, return both clearly. Include % reductions, baseline year/scenario (e.g. BAU), and any quantitative values (e.g. MtCO2e). Be concise. Always cite the relevant chunks using [Doc ID: <id>, Chunk ID: <id>]. If a target is not found, respond: 'The target is not clearly specified in the provided chunks.'
role: user
content: Context:
[Doc ID: cameroon_french_20220601, Chunk ID: 677] 
 'CO2, soit 12 % des émissions dans le scenario BAU 2030.'
[Doc ID: cameroon_french_20220601, Chunk ID: 898] 
 '+0,6° C en 2025 à +3,6° C en 2100.'
[Doc ID: cameroon_french_20220601, Chunk ID: 569] 
 'Scénario "with measures" 2030'
[Doc ID: cameroon_french_20220601, Chunk ID: 629] 
 'électrique en 2035 »'

Question: What is the official 2030 emissions reduction target for Cameroon, based on the provided docume

{'Cameroon': [{'role': 'system',
   'content': "You are a climate policy expert. Based only on the provided document chunks, extract the country’s 2030 emissions reduction target. If both unconditional and conditional targets exist, return both clearly. Include % reductions, baseline year/scenario (e.g. BAU), and any quantitative values (e.g. MtCO2e). Be concise. Always cite the relevant chunks using [Doc ID: <id>, Chunk ID: <id>]. If a target is not found, respond: 'The target is not clearly specified in the provided chunks.'"},
  {'role': 'user',
   'content': 'Context:\n[Doc ID: cameroon_french_20220601, Chunk ID: 677] \n \'CO2, soit 12 % des émissions dans le scenario BAU 2030.\'\n[Doc ID: cameroon_french_20220601, Chunk ID: 898] \n \'+0,6° C en 2025 à +3,6° C en 2100.\'\n[Doc ID: cameroon_french_20220601, Chunk ID: 569] \n \'Scénario "with measures" 2030\'\n[Doc ID: cameroon_french_20220601, Chunk ID: 629] \n \'électrique en 2035 »\'\n\nQuestion: What is the official 2030 emission

__Re-running our API queries__

In [470]:
llm_responses_empty_countries = {}
for country, message in second_loop_country_message_directory.items():
    print(f"Processing country: {country}")
    flattened_text = "".join([m["content"] for m in message])
    tokenized = tokenizer(flattened_text, return_tensors="pt", truncation = True)
    num_tokens = len(tokenized["input_ids"][0])
    
    print(f"[{country}] Token count: {num_tokens}")
    response = client.chat.completions.create(
        model=llm_model,
        max_tokens=1024,
        temperature=0.6,
        top_p=0.9,
        extra_body={
            "top_k": 50
        },
        messages=message
    )
    print(f"API response returned for: {country}")

    llm_responses_empty_countries[country] = response.choices[0].message.content

Processing country: Cameroon
[Cameroon] Token count: 297
API response returned for: Cameroon
Processing country: Papua New Guinea
[Papua New Guinea] Token count: 475
API response returned for: Papua New Guinea
Processing country: Sri Lanka
[Sri Lanka] Token count: 496
API response returned for: Sri Lanka
Processing country: Uzbekistan
[Uzbekistan] Token count: 512
API response returned for: Uzbekistan
Processing country: Saudi Arabia
[Saudi Arabia] Token count: 318
API response returned for: Saudi Arabia
Processing country: Cuba
[Cuba] Token count: 458
API response returned for: Cuba
Processing country: Maldives
[Maldives] Token count: 512
API response returned for: Maldives
Processing country: Ghana
[Ghana] Token count: 410
API response returned for: Ghana
Processing country: State of Palestine
[State of Palestine] Token count: 512
API response returned for: State of Palestine
Processing country: China
[China] Token count: 512
API response returned for: China
Processing country: Domin

In [471]:
llm_responses_empty_countries

{'Cameroon': 'The target is not clearly specified in the provided chunks.',
 'Papua New Guinea': 'The target is not clearly specified in the provided chunks.',
 'Sri Lanka': 'The target is not clearly specified in the provided chunks.',
 'Uzbekistan': 'The target is not clearly specified in the provided chunks.',
 'Saudi Arabia': 'The target is not clearly specified in the provided chunks.',
 'Cuba': 'The target is not clearly specified in the provided chunks.',
 'Maldives': 'The target is not clearly specified in the provided chunks.',
 'Ghana': 'The target is not clearly specified in the provided chunks.',
 'State of Palestine': 'The target is not clearly specified in the provided chunks.',
 'China': 'The target is not clearly specified in the provided chunks.',
 'Dominica': 'The official 2030 emissions reduction target for Dominica is a 45% reduction below 2014 levels [Doc ID: dominica_english_20220701, Chunk ID: 250].',
 'Angola': 'The target is not clearly specified in the provide

In [472]:
still_non_retrieved_countries = []
redeemed_countries = []
for key, value in llm_responses_empty_countries.items():
    if value == 'The target is not clearly specified in the provided chunks.':
        still_non_retrieved_countries.append(key)
    
    else:
        redeemed_countries.append(key)

print(len(still_non_retrieved_countries), len(redeemed_countries))

48 3


In [477]:
pickle.dump(llm_responses_empty_countries, open("llm_responses_non_retrieved_countries.pkl", "wb")) 
pickle.dump(llm_responses, open("llm_responses.pkl", "wb"))
pickle.dump(complete_embeddings_df, open("complete_embeddings_df.pkl", "wb"))

##### ======= ADDITIONAL: INCOMPLETE WORK =========
__I had also wanted to implement an additional set of retrieved chunks for our documents in spanish and french by constructing queries in those native language and then running a similarity search through our chunks database. I didn't get time to implement this, but have left in the NB as a WIP to finish personally later__

In [79]:
query = text("""
    SELECT DISTINCT 
        split_part(doc_id, '_', array_length(string_to_array(doc_id, '_'), 1) - 1) AS language
    FROM doc_chunks
""")

with engine.connect() as conn:
    languages = [row[0] for row in conn.execute(query).fetchall()]

In [80]:
languages

['english', 'french', 'spanish']

In [144]:
language_queries = {
    "french": "Quel est l’objectif de réduction des émissions visé par le pays d’ici 2030 ?",
    "spanish": "¿Cuál es el objetivo de reducción de emisiones que persigue el país para 2030?"
}

query_embeddings = {
    lang: get_pooled_embeddings(tokenize_for_transformer_climate_roberta(q), climate_roberta_model)
    for lang, q in language_queries.items()
}

In [149]:
query_embeddings['english']

(768,)

In [ ]:
all_lang_results = []

for lang in language_queries.keys():
    query = text("""
        SELECT 
            doc_chunks.doc_id,
            doc_chunks.chunk_index,
            doc_chunks.content,
            documents.country,
            (doc_chunks.embedding <=> :embedding) AS cosine_distance
        FROM doc_chunks
        LEFT JOIN documents ON doc_chunks.doc_id = documents.doc_id
        WHERE 
            split_part(doc_chunks.doc_id, '_', array_length(string_to_array(doc_chunks.doc_id, '_'), 1) - 1) = :lang
            AND doc_chunks.content ILIKE '%2030%'
            AND (doc_chunks.embedding <=> :embedding) <= :distance_threshold
        ORDER BY cosine_distance ASC
        LIMIT 10
    """)

    with engine.connect() as conn:
        results = conn.execute(query, {
            "embedding": query_embeddings[lang],
            "lang": lang,
            "distance_threshold": 0.05
        }).fetchall()

    df = pd.DataFrame(results, columns=[
        "doc_id", "chunk_index", "content", "country", "cosine_distance"
    ])
    df["language"] = lang
    df["cosine_similarity"] = 1 - df["cosine_distance"]
    all_lang_results.append(df)

In [ ]:
multilingual_df = pd.concat(all_lang_results, ignore_index=True)